<a href="https://colab.research.google.com/github/Sarthakk0/21BAI1165/blob/main/Music_Generation_DL_Project_SarthakGupta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split

In [ ]:
def read_files(file):

    notes = []
    notes_to_parse = None

    try:

        midi = converter.parse(file)
    except:
        print(f"error in {file}")
        return None

    else:

        #Seprating all Instruments fromthe file
        instrmt = instrument.partitionByInstrument(midi)

        for part in instrmt.parts:

            #Fetching only Piano
            if 'Piano' in str(part):
                notes_to_parse = part.recurse()

            for element in notes_to_parse:
                if isinstance(element,note.Note):

                    notes.append(str(element.pitch))

                elif isinstance(element,chord.Chord):

                    notes.append('.'.join(str(n) for n in element.normalOrder))
        return notes

In [ ]:

file_path = ["schubert"]
all_files = glob.glob('/content/*.mid',recursive=True)

notes_array = [read_files(i) for i in tqdm(all_files,position=0,leave=True)]

# notes_array = []
# for i in tqdm(all_files,position=0,leave=True):
#     print(i)
#     notes_array.append(read_files(i))

100%|██████████| 29/29 [00:45<00:00,  1.57s/it]


In [ ]:
#Removing none from notes_aaray

updated_notes_array = []
for i in list(notes_array):
    if i != None:
        updated_notes_array.append(i)




In [ ]:
#Let’s check how many unique notes we have and what is the distribution of these notes.


notes = sum(updated_notes_array,[])
unique_notes = list(set(notes))
print("Unique Notes:",len(unique_notes))

#notes with their frequency

freq = dict(map(lambda x : (x,notes.count(x)),unique_notes))

#threshold freq

for i in range(30,100,20):
    print(i,":",len(list(filter(lambda x:x[1]>i,freq.items()))))

Unique Notes: 345
30 : 204
50 : 180
70 : 163
90 : 152


In [ ]:
#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))
#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in list(updated_notes_array)]


print(new_notes)
print(freq_notes)

[['8.0.3', 'G#3', '5.8', 'C5', 'C#4', 'B-4', '5.8', 'C#4', '1.5', 'F5', '8.0', 'C4', 'F4', 'E-5', '8.0', 'E-4', 'C4', 'C4', 'E-4', 'G#3', 'G#4', 'E-3', 'C5', 'C3', '3.7.10', '8.0.3', '3.8', '10.3', '3.7', '5.10', '2.8', 'B-4', '10.3', '3.7', 'B-4', '10.3', '8.0.3', 'G#3', 'C5', '5.8', 'C#4', 'B-4', '5.8', 'C#4', '1', '7.0', 'F5', '0', 'E5', '7.0', '0', '0.4.7', '0.4', '10.3', '8.0.3', '3.8', '10.1.5', 'C#3', '1.7', 'C5', 'E-3', 'B-4', '7.10.1', '3.8', '8.0', '3.8', '8.11.2', '4.8.11', 'D4', '8.11', 'E3', '8.11', 'E-4', '8.11', 'E-3', '8.11', '4.8.11', 'E4', '8.11', 'C#3', '8.11', 'E-4', '8.11', 'B2', '8.11', '8.11.3', '8.11', 'B-4', '4.8', 'C#3', '8.10', 'E5', 'B-4', 'E-5', '4.8', 'B-2', '8.10', 'C#5', 'B4', 'B2', '10.1.4', 'C#3', 'E-4', '8.11', 'E-3', '7.10', 'E-4', 'E-3', '8.11.2', '4.8.11', 'D4', '8.11', 'E3', '8.11', 'E-4', '8.11', 'E-3', '8.11', '4.8.11', 'E4', '8.11', 'C#3', '8.11', 'E-4', '8.11', 'B2', '8.11', '4.8.11', '4.8.11', 'B4', 'E4', 'G#4', '4.8.11', 'E5', 'B4', 'B4', 'E

In [ ]:
#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))
#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

In [ ]:
timesteps = 50

x = []
y = []

for i in new_notes:
    for j in range(0,len(i)-timesteps):
        #input will be the current index + timestep
        #output will be the next index after timestep

        inp = i[j:j+timesteps]
        out = i[j+timesteps]

        x.append(list(map(lambda x:note2ind[x],inp)))

        y.append(note2ind[out])

x_new = np.array(x)
y_new = np.array(y)

In [ ]:
x_new

array([[156,   3, 175, ..., 149, 112,  39],
       [  3, 175, 142, ..., 112,  39, 149],
       [175, 142, 177, ...,  39, 149,  72],
       ...,
       [ 88,  88, 172, ...,   3, 116, 121],
       [ 88, 172,  88, ..., 116, 121,   2],
       [172,  88,  67, ..., 121,   2, 156]])

In [ ]:
#reshape input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))
#split the input and value into training and testing sets
#80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

In [ ]:
model = Sequential()

model.add(LSTM(256,return_sequences=True,input_shape = (x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 182)               46774     
                                                                 
Total params: 902070 (3.44 MB)
Trainable params: 902070 

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])


model.fit(
    x_train,y_train,
    batch_size=128,
    epochs = 80,
    validation_data=(x_test,y_test)

    )

Epoch 1/80
489/489 [==============================] - 15s 20ms/step - loss: 4.7662 - accuracy: 0.0303 - val_loss: 4.6453 - val_accuracy: 0.0460
Epoch 2/80
489/489 [==============================] - 9s 18ms/step - loss: 4.5040 - accuracy: 0.0577 - val_loss: 4.4230 - val_accuracy: 0.0714
Epoch 3/80
489/489 [==============================] - 9s 18ms/step - loss: 4.2659 - accuracy: 0.0857 - val_loss: 4.1553 - val_accuracy: 0.1029
Epoch 4/80
489/489 [==============================] - 9s 19ms/step - loss: 4.0087 - accuracy: 0.1175 - val_loss: 3.9568 - val_accuracy: 0.1273
Epoch 5/80
489/489 [==============================] - 9s 19ms/step - loss: 3.7475 - accuracy: 0.1500 - val_loss: 3.7715 - val_accuracy: 0.1510
Epoch 6/80
489/489 [==============================] - 9s 18ms/step - loss: 3.5103 - accuracy: 0.1853 - val_loss: 3.6030 - val_accuracy: 0.1794
Epoch 7/80
489/489 [==============================] - 9s 18ms/step - loss: 3.3026 - accuracy: 0.2138 - val_loss: 3.4635 - val_accuracy: 0.193

In [ ]:
model.save("Try1_190224")

In [ ]:
model.save("Try1_190224.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
index = np.random.randint(0,len(x_test)-1)

In [ ]:
music_pattern = x_test[index]
out_pred=[] #it will store predicted notes

In [ ]:
for i in range(200):
 #reshape the music pattern
 music_pattern = music_pattern.reshape(1,len(music_pattern),1)
 #get the maximum probability value from the predicted output
 pred_index = np.argmax(model.predict(music_pattern))
 #get the note using predicted index and
 #append to the output prediction list
 out_pred.append(ind2note[pred_index])
 music_pattern = np.append(music_pattern,pred_index)
 #update the music pattern with one timestep ahead
 music_pattern = music_pattern[1:]

1/1 [==============================] - 0s 21ms/step


In [ ]:
output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
      i_curr_note=int(current_note)
      #cast the current note to Note object and
      #append the current note
      new_note = note.Note(i_curr_note)
      new_note.storedInstrument = instrument.Piano()
      notes.append(new_note)

    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)

  else:
    #cast the pattern to Note object apply the offset and
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='pred_music.mid')

'pred_music.mid'